In [3]:
import pandas as pd
import gmaps
import requests
from config import google_api
import json

In [4]:
city_df = pd.read_csv('weather_data/cities.csv')
city_df.set_index("City_ID")

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
City_ID,,,,,,,,,
0,Saskylakh,RU,2022-10-12 00:03:21,71.9167,114.0833,26.37,96,98,5.93
1,Hobart,AU,2022-10-12 00:02:08,-42.8794,147.3294,64.06,70,75,14.97
2,Avarua,CK,2022-10-12 00:02:14,-21.2078,-159.7750,78.85,78,75,20.71
3,Albany,US,2022-10-12 00:00:26,42.6001,-73.9662,56.23,84,2,4.05
4,Port Elizabeth,ZA,2022-10-12 00:03:22,-33.9180,25.5701,67.32,87,96,4.00
...,...,...,...,...,...,...,...,...,...
557,Cheremukhovo,RU,2022-10-12 00:19:52,60.3484,59.9851,39.47,85,76,2.57
558,Terrenate,MX,2022-10-12 00:19:52,19.4833,-97.9167,53.10,86,99,6.15
559,General Roca,AR,2022-10-12 00:19:53,-39.0333,-67.5833,65.62,35,96,28.70


In [5]:
len(city_df["Max Temp"])

562

In [6]:
gmaps.configure(api_key=google_api)


In [7]:

def heat_method(measure):
    locations = city_df[['Lat','Lng']]
    weight = measure
    fig = gmaps.figure(center=(30.,31.),zoom_level=2)
    heat_layer = gmaps.heatmap_layer(locations,weights=[max(i,0) for i in weight],dissipating=False,point_radius=4)
    fig.add_layer(heat_layer)
    fig
    return fig

In [8]:
heat_method(city_df["Max Temp"])

Figure(layout=FigureLayout(height='420px'))

In [9]:
heat_method(city_df["Humidity"])

Figure(layout=FigureLayout(height='420px'))

In [10]:
heat_method(city_df["Cloudiness"])

Figure(layout=FigureLayout(height='420px'))

In [11]:
heat_method(city_df['Wind Speed'])

Figure(layout=FigureLayout(height='420px'))

In [13]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [14]:
preferred_cities_df = city_df.loc[(city_df["Max Temp"] <= max_temp) & (city_df["Max Temp"] >= min_temp)]
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID       149
City          149
Country       149
Date          149
Lat           149
Lng           149
Max Temp      149
Humidity      149
Cloudiness    149
Wind Speed    149
dtype: int64

In [15]:
hotel_df = preferred_cities_df[['City','Country','Max Temp','Lat','Lng']].copy()
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Avarua,CK,78.85,-21.2078,-159.7750,
20,Oakdale,US,75.42,44.9630,-92.9649,
21,Butaritari,KI,82.20,3.0707,172.7902,
28,San Pedro,PH,83.32,14.3500,121.0167,
38,Atuona,PF,77.40,-9.8000,-139.0333,
...,...,...,...,...,...,...
541,Oranjestad,AW,84.15,12.5240,-70.0270,
545,Point Pedro,LK,78.58,9.8167,80.2333,
549,Bairiki,KI,82.40,1.3292,172.9752,
551,Mindelo,CV,78.35,16.8901,-24.9804,


In [16]:
params = {
   'radius' : 5000,
   'key' : google_api,
   'location' : (47.6062,-122.3320),
   'types' : 'lodging'
   }
results = {}

In [21]:
for index, row in hotel_df[:10].iterrows():
    lat = row["Lat"]
    lng = row['Lng']

    params['location'] = f"{lat},{lng}"
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
        hotels = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel Not Found, skipping")
    
    

City          149
Country       149
Max Temp      149
Lat           149
Lng           149
Hotel Name    149
dtype: int64

In [30]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))